# Environment
## States
There are five rooms and two areas in which the robot could enter. Each of them is a state.

## Actions
From every room, there are several doors that would connect the room to another room or area. The robot's action is to choose one of these doors and go through it. Note that some doors have only one room, and areas could be accessed via one door that connects them to a room.

## Rewards
Area numbered 5 has a reward of 10, and Area numbered 6 has a penalty of -10. Every action (entering rooms) has a penalty of -1.

## Goal State
The goal state is area number 5. The robot is supposed to start from room number 2 and reach area number 5.


# Environment Implementation
We use an adjacency list for representing the graph and a list of dictionaries for Q, as you can see here. Rewards (and penalties) are stored in an adjacency matrix, and edge weights represent the rewards. Also, the initial value for Q is zero.

# Learning
This part is implemented by two loops. The first is for episodes that would run the games `n_episodes` times. The second is responsible for one game and would end if the agent ends up in a trap or goal or the number of steps exceeds a specific number (Here, it is 1000).

In each step, first, we would choose an action for the agent based on its state. This decision is made based on `epsilon`, the probability of making a random decision, and if it does not make an arbitrary decision, it will choose the best action (node). This algorithm is called *Epsilon-greedy*.

After selecting the next node, we would update Q based on Q learning formula (specified in `calculate_new_Q`). Then we change the agent state and repeat these steps.

In [95]:
import random


class Graph:
	def __init__(self, adj, goals, traps, n_episods, learning_rate, discount_factor, epsilon, goal_reward, trap_penalty, movement_penalty):
		self.adj = adj
		self.learning_rate = learning_rate
		self.discount_factor = discount_factor
		self.epsilon = epsilon
		self.n_episods = n_episods
		self.goals = goals
		self.traps = traps

		self.rewards = []
		for i in range(len(self.adj)):
			self.rewards.append([])
			for j in range(len(self.adj)):
				if j in self.adj[i]:
					if j in goals:
						self.rewards[-1].append(goal_reward)
					elif j in traps:
						self.rewards[-1].append(trap_penalty)
					else:
						self.rewards[-1].append(movement_penalty)
				else:
					self.rewards[-1].append(0)

		self.Q = []
		for node1 in self.adj:
			self.Q.append(dict())
			for node2 in node1:
				self.Q[-1][node2] = 0

	def learn(self, first_node):
		for _ in range(self.n_episods):
			step = 0
			current_node = first_node
			while step < 10000 and current_node not in self.goals and current_node not in self.traps:
				next_node = self.choose_action(current_node)
				self.calculate_new_Q(current_node, next_node)
				current_node = next_node
				step += 1

	def calculate_new_Q(self, current_node, next_node):
		self.Q[current_node][next_node] = (1 - self.learning_rate) * self.Q[current_node][next_node] + \
			self.learning_rate * \
			(self.rewards[current_node][next_node] +
			 self.discount_factor * max(self.Q[next_node].values()))

	def choose_action(self, current_node):
		if random.random() < self.epsilon:
			return random.choice(self.adj[current_node])
		else:
		    return random.choice([key for key in self.Q[current_node]
                        if self.Q[current_node][key] == max(self.Q[current_node].values())])

	def play(self, first_node):
		step = 0
		current_node = first_node
		while step < 10 and current_node not in self.goals and current_node not in self.traps:
			print('Step:', step, '\tNode:', current_node)
			next_node = random.choice([key for key in self.Q[current_node]
                              if self.Q[current_node][key] == max(self.Q[current_node].values())])
			current_node = next_node
			step += 1
		print('Step:', step, '\tNode:', current_node)


# Testing
This part the agent would play once and only choose the best action.

In [96]:
dj = [
	[4],
	[3, 5],
	[3],
	[1, 2, 4],
	[0, 3, 6],
	[1],
	[4]
]

goals = [5]
traps = [6]

goal_reward = 10
trap_penalty = -10

movement_penalty = -1

Here we test the algorithm with 5 different parameters

## First
As you could see it found the best path.

In [97]:
n_episods = 10

learning_rate = 0.8

discount_factor = 0.8

epsilon = 0.8

graph = Graph(adj, goals, traps, n_episods, learning_rate, discount_factor,
              epsilon, goal_reward, trap_penalty, movement_penalty)

graph.learn(2)
print(list(enumerate(graph.Q)))
graph.play(2)


[(0, {4: -0.8}), (1, {3: 4.52679261093888, 5: 9.999872}), (2, {3: 4.59009217143505}), (3, {1: 6.9994054254592, 2: 2.6526596108721856, 4: -1.6348160000000003}), (4, {0: -0.8, 3: -1.7497600000000002, 6: -9.92}), (5, {1: 0}), (6, {4: 0})]
Step: 0 	Node: 2
Step: 1 	Node: 3
Step: 2 	Node: 1
Step: 3 	Node: 5


## Second
As you could see it found the best path.

In [98]:
n_episods = 3

learning_rate = 0.9

discount_factor = 0.5

epsilon = 0.5

graph = Graph(adj, goals, traps, n_episods, learning_rate, discount_factor,
              epsilon, goal_reward, trap_penalty, movement_penalty)

graph.learn(2)
print(list(enumerate(graph.Q)))
graph.play(2)

[(0, {4: -0.9}), (1, {3: 0, 5: 9.9}), (2, {3: -1.4048999999999998}), (3, {1: 3.06, 2: -1.3455000000000001, 4: -0.99}), (4, {0: -0.9, 3: -1.305, 6: -9.0}), (5, {1: 0}), (6, {4: 0})]
Step: 0 	Node: 2
Step: 1 	Node: 3
Step: 2 	Node: 1
Step: 3 	Node: 5


## Third
Here as you could see because of low learning rate, low discount factor the agent was unable to learn what action are good because the rewards had very little effect.

In [99]:
n_episods = 3

learning_rate = 0.1

discount_factor = 0.1

epsilon = 0.8

graph = Graph(adj, goals, traps, n_episods, learning_rate, discount_factor,
              epsilon, goal_reward, trap_penalty, movement_penalty)

graph.learn(2)
print(list(enumerate(graph.Q)))
graph.play(2)


[(0, {4: 0}), (1, {3: 0, 5: 1.9}), (2, {3: -0.3458}), (3, {1: -0.18000000000000002, 2: -0.10189999999999999, 4: -0.19}), (4, {0: 0, 3: -0.1, 6: -1.0}), (5, {1: 0}), (6, {4: 0})]
Step: 0 	Node: 2
Step: 1 	Node: 3
Step: 2 	Node: 2
Step: 3 	Node: 3
Step: 4 	Node: 2
Step: 5 	Node: 3
Step: 6 	Node: 2
Step: 7 	Node: 3
Step: 8 	Node: 2
Step: 9 	Node: 3
Step: 10 	Node: 2


## Forth
Here as you could see even though learning rate and discount factor is low because number of episodes is relatively high, the agent had enough chance to explore the environment and experience different action enough to find the best path.

In [100]:
n_episods = 8

learning_rate = 0.1

discount_factor = 0.1

epsilon = 0.8

graph = Graph(adj, goals, traps, n_episods, learning_rate, discount_factor,
              epsilon, goal_reward, trap_penalty, movement_penalty)

graph.learn(2)
print(list(enumerate(graph.Q)))
graph.play(2)


[(0, {4: -0.27626619}), (1, {3: -0.42430913740000004, 5: 3.439}), (2, {3: -0.8177032694358313}), (3, {1: -0.4856000220000002, 2: -0.546942358302851, 4: -0.5269035900000001}), (4, {0: -0.273837101, 3: -0.27371, 6: -3.439}), (5, {1: 0}), (6, {4: 0})]
Step: 0 	Node: 2
Step: 1 	Node: 3
Step: 2 	Node: 1
Step: 3 	Node: 5


## Fifth
In this example because the epsilon is very low we don't give a chance to the agent to explore the environment.

In [101]:
n_episods = 8

learning_rate = 0.1

discount_factor = 0.1

epsilon = 0.01

graph = Graph(adj, goals, traps, n_episods, learning_rate, discount_factor,
              epsilon, goal_reward, trap_penalty, movement_penalty)

graph.learn(2)
print(list(enumerate(graph.Q)))
graph.play(2)


[(0, {4: -0.34921769933579005}), (1, {3: 0, 5: 5.217031}), (2, {3: -0.7329255530701002}), (3, {1: -0.3720087000000001, 2: -0.3595746399790001, 4: -0.41477710100000004}), (4, {0: -0.34916746582429004, 3: -0.35229170347210004, 6: -1.0}), (5, {1: 0}), (6, {4: 0})]
Step: 0 	Node: 2
Step: 1 	Node: 3
Step: 2 	Node: 2
Step: 3 	Node: 3
Step: 4 	Node: 2
Step: 5 	Node: 3
Step: 6 	Node: 2
Step: 7 	Node: 3
Step: 8 	Node: 2
Step: 9 	Node: 3
Step: 10 	Node: 2
